#### 1. Запросить курсы любой валюты за указанный период. Сохранить полученные данные в базу данных Mongo.

In [1]:
from zeep import Client
from pprint import pprint
from pymongo import MongoClient, errors

In [2]:
def create_db(database_name, index, unique=True):
    try:
        client = MongoClient('mongodb://127.0.0.1:27017')
        db = client[database_name]
        if index:
            db[database_name].create_index(index, unique=unique)
        return db[database_name]
    except:
        return False


def add_to_db(database, document):
    try:
        database.insert_one(document)
        return True
    except errors.DuplicateKeyError:
        return False


def enum_valutes(seld, database_name, index, unique=True):
    try:
        url = 'http://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?WSDL'
        client = Client(url)
        result = client.service.EnumValutes(seld)
    
        db = create_db(database_name, index, unique)
    
        for value in result._value_1._value_1:
            doc = {
                'Vcode': value['EnumValutes']['Vcode'],
                'Vname': value['EnumValutes']['Vname'],
                'VEngname': value['EnumValutes']['VEngname'],
                'VcharCode': value['EnumValutes']['VcharCode']
            }
            add_to_db(db, doc)
        return db
    except:
        return False
    

def get_currency_code(database, vcharcode):
    try:
        vcode = database.find_one({'VcharCode': vcharcode})['Vcode']
        return vcode
    except:
        return False

    
def get_curs_dynamic(database_name, index, currency_database, vcharcode, from_date, to_date):
    try:
        valuta_code = get_currency_code(currency_database, vcharcode)
        
        url = 'http://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?WSDL'
        client = Client(url)
        result = client.service.GetCursDynamic(from_date, to_date, valuta_code)
        
        db = create_db(database_name, index)
        
        for value in result._value_1._value_1:
            doc = {
                'VcharCode': vcharcode,
                'CursDate': value['ValuteCursDynamic']['CursDate'],
                'Vcurs': float(value['ValuteCursDynamic']['Vcurs'].to_eng_string())
            }
            add_to_db(db, doc)
        return db
    except err:
        print(err)

In [3]:
currency_db = enum_valutes(False, 'currency', 'VcharCode', True)

In [4]:
currency_db.find_one({'VcharCode': 'USD'})

{'_id': ObjectId('5d5ed2e8cc551af5dce19f64'),
 'Vcode': 'R01235              ',
 'Vname': 'Доллар США                                                                                                                                                                                                                                                    ',
 'VEngname': 'US Dollar                                                                                                                                                                                                                                                     ',
 'VcharCode': 'USD'}

In [5]:
currency_rate_db = get_curs_dynamic('currency_rate', False, currency_db, 'USD', '2019-08-01', '2019-08-21')

In [6]:
currency_rate_db.find_one({'VcharCode': 'USD'})

{'_id': ObjectId('5d5ee3e19ea2f27ec6a820e7'),
 'VcharCode': 'USD',
 'CursDate': datetime.datetime(2019, 7, 31, 21, 0),
 'Vcurs': 63.4172}

#### 3. Сделать вывод информативным: Валюту <Валюта> выгодно было купить <Дата>, а продать <Дата>. Прибыль: <Разница в цене валюты>.

#### 4. Доработать приложение по вакансиям. При наличии вакансии с зарплатой отличной от руб. приложение должно осущетсвлять автоматический перевод по курсу валюты на сегодняшний день. Информацию получать через SOAP-сервис центробанка.